# XML/HTML examples

## HTML

Python has numerous libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples are [lxml](#lxml), [Beautiful Soup](beautifulsoup.ipynb) and html5lib. While lxml is generally comparatively much faster, the other libraries are better at handling corrupted HTML or XML files.

pandas has a built-in function, `read_html`, which uses libraries like lxml, html5lib and Beautiful Soup to automatically parse tables from HTML files as DataFrame objects. These have to be installed additionally. With [Spack](../../../productive/envs/spack/index.rst) you can provide lxml, BeautifulSoup and html5lib in your kernel:

```console
$ spack env activate python-38
$ spack install py-lxml@4.6.3%gcc@11.2.0 py-beautifulsoup4@4.10.0%gcc@11.2.0~html5lib~lxml py-html5lib@1.1%gcc@11.2.0
```

Alternatively, you can install BeautifulSoup with other package managers, for example

```console
$ pipenv install lxml beautifulsoup4 html5lib
```

To show how this works, I use an HTML file from Wikipedia that gives an overview of different serialisation formats.

In [1]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/Comparison_of_data-serialization_formats')

The `pandas.read_html` function has a number of options, but by default it looks for and tries to parse all table data contained in `<table>` tags. The result is a list of DataFrame objects:

In [2]:
len(tables)

3

In [3]:
formats = tables[0]

formats.head()

,Name,Creator-maintainer,Based on,Standardized?[definition needed],Specification,Binary?,Human-readable?,Supports references?e,Schema-IDL?,Standard APIs,Supports zero-copy operations
0,Apache Avro,Apache Software Foundation,NaN,No,Apache Avro™ Specification,Yes,Partialg,NaN,Built-in,"C, C#, C++, Java, PHP, Python, Ruby",NaN
1,Apache Parquet,Apache Software Foundation,NaN,No,Apache Parquet,Yes,No,No,NaN,"Java, Python, C++",No
2,ASN.1,"ISO, IEC, ITU-T",NaN,Yes,ISO/IEC 8824 / ITU-T X.680 (syntax) and ISO/IE...,"BER, DER, PER, OER, or custom via ECN","XER, JER, GSER, or custom via ECN",Yesf,Built-in,NaN,OER
3,Bencode,"Bram Cohen (creator)BitTorrent, Inc. (maintainer)",NaN,De facto as BEP,Part of BitTorrent protocol specification,"Except numbers and delimiters, being ASCII",No,No,No,No,No
4,Binn,Bernardo Ramos,JSON (loosely),No,Binn Specification,Yes,No,No,No,No,Yes


From here we can do some [data cleansing and analysis](../../../clean-prep/index.rst), such as the number of different schema IDLs:

In [4]:
formats['Schema-IDL?'].value_counts()

No                                                                                        17
Built-in                                                                                   4
Yes                                                                                        4
CDDL                                                                                       1
Partial(Signature strings)                                                                 1
XML Schema                                                                                 1
XML schema                                                                                 1
Ion schema                                                                                 1
Partial(JSON Schema Proposal, ASN.1 with JER, Kwalify, Rx, Itemscript Schema), JSON-LD     1
Schema WD                                                                                  1
?                                                                     

## XML

pandas has a function `read_xml`, which makes reading XML files very easy:

In [5]:
pd.read_xml('books.xml')

,id,title,language,author,license,date
0,1,Python basics,en,Veit Schiele,BSD-3-Clause,2021-10-28
1,2,Jupyter Tutorial,en,Veit Schiele,BSD-3-Clause,2019-06-27
2,3,Jupyter Tutorial,de,Veit Schiele,BSD-3-Clause,2020-10-26
3,4,PyViz Tutorial,en,Veit Schiele,BSD-3-Clause,2020-04-13


### `lxml`

Alternatively, `lxml.objectify` can be used first to parse XML files. In doing so, we get a reference to the root node of the XML file with `getroot`:

In [6]:
from lxml import objectify

parsed = objectify.parse(open('books.xml'))
root = parsed.getroot()

In [7]:
books = []

for element in root.book:
    data = {}
    for child in element.getchildren():
        data[child.tag] = child.pyval
    books.append(data)

In [8]:
pd.DataFrame(books)

,title,language,author,license,date
0,Python basics,en,Veit Schiele,BSD-3-Clause,2021-10-28
1,Jupyter Tutorial,en,Veit Schiele,BSD-3-Clause,2019-06-27
2,Jupyter Tutorial,de,Veit Schiele,BSD-3-Clause,2020-10-26
3,PyViz Tutorial,en,Veit Schiele,BSD-3-Clause,2020-04-13
